In [ ]:
!pip install -q --upgrade pip setuptools
!pip install -q "numpy~=1.26.4"
!pip install -q ultralytics tqdm requests PyYAML
!pip install -q onnx==1.17.0 onnxruntime==1.22.0 onnxconverter-common==1.14.0


In [ ]:
!mkdir -p /content/my_coco_data/images
!unzip -q /content/val2017.zip -d /content/my_coco_data/images/
!unzip -q /content/annotations_trainval2017.zip -d /content/my_coco_data

In [ ]:
import torch
from ultralytics import YOLO
from pathlib import Path
import os
import shutil
import json
from tqdm import tqdm
import time
import numpy as np

try:
    import onnx
    import onnxruntime as ort
    from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantFormat, QuantType
except ImportError:
    print("ONNX libraries not found, ensure they are installed in the next cell.")

print(f"Torch: {torch.__version__}, NumPy: {np.__version__}")
DATASET_ROOT_DIR = "/content/my_coco_data"
MODEL_PT = 'yolo11m.pt'
EXPORT_DIR_BASE = 'exported_models_coco_val2017'

IMAGES_SUBDIR_RELATIVE = "images/val2017"
JSON_ANNOTATION_SUBDIR_RELATIVE = "annotations/instances_val2017.json"
LABELS_SUBDIR_RELATIVE = "labels/val2017"

ABS_IMAGES_DIR = Path(DATASET_ROOT_DIR) / IMAGES_SUBDIR_RELATIVE
ABS_JSON_ANNOTATION_FILE = Path(DATASET_ROOT_DIR) / JSON_ANNOTATION_SUBDIR_RELATIVE
ABS_LABELS_DIR = Path(DATASET_ROOT_DIR) / LABELS_SUBDIR_RELATIVE

DATASET_YAML_NAME = "coco_val2017_for_yolo.yaml"
ABS_DATASET_YAML_PATH = f"/content/{DATASET_YAML_NAME}"

COCO_CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
]

In [ ]:
def convert_coco_json_to_yolo_txt(json_annotation_path_abs, image_dir_path_abs, output_label_dir_path_abs, class_names_list):
    if not json_annotation_path_abs.is_file():
        print(f"ERROR: JSON Annotation file not found: {json_annotation_path_abs}")
        return False
    os.makedirs(output_label_dir_path_abs, exist_ok=True)
    try:
        with open(json_annotation_path_abs, 'r') as f: data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON {json_annotation_path_abs}: {e}"); return False
    if not all(k in data for k in ['categories', 'images', 'annotations']):
        print(f"ERROR: JSON {json_annotation_path_abs} missing keys."); return False

    coco_id_to_name = {cat['id']: cat['name'] for cat in data['categories']}
    name_to_target_id = {name: i for i, name in enumerate(class_names_list)}
    coco_cat_id_to_target_id = {cid: name_to_target_id[cname] for cid, cname in coco_id_to_name.items() if cname in name_to_target_id}
    image_info = {img['id']: {'fn': img['file_name'], 'w': img['width'], 'h': img['height']} for img in data['images']}
    img_ann = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        if img_id not in img_ann: img_ann[img_id] = []
        img_ann[img_id].append(ann)
    created_count = 0
    for img_id, anns in tqdm(img_ann.items(), desc=f"Converting {json_annotation_path_abs.name}", leave=False):
        if img_id not in image_info: continue
        img_d = image_info[img_id]
        if img_d['w'] == 0 or img_d['h'] == 0: continue
        with open(output_label_dir_path_abs / f"{Path(img_d['fn']).stem}.txt", 'w') as f_out:
            for ann_d in anns:
                if ann_d['category_id'] in coco_cat_id_to_target_id:
                    cls_id = coco_cat_id_to_target_id[ann_d['category_id']]
                    x, y, w, h = ann_d['bbox']
                    xc, yc = (x + w / 2) / img_d['w'], (y + h / 2) / img_d['h']
                    wn, hn = w / img_d['w'], h / img_d['h']
                    f_out.write(f"{cls_id} {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}\n")
            created_count += 1
    if created_count == 0 and img_ann: print("WARNING: No labels created.")
    return True

In [ ]:
if os.path.exists(ABS_LABELS_DIR): shutil.rmtree(ABS_LABELS_DIR)
_paths_to_check_for_cache = [ABS_LABELS_DIR.parent, ABS_LABELS_DIR, Path(ABS_DATASET_YAML_PATH).parent]
for pth in _paths_to_check_for_cache:
    if pth.exists():
        for cache_file in pth.glob('*.cache'):
            if cache_file.is_file() and (Path(ABS_DATASET_YAML_PATH).stem in cache_file.name or ABS_LABELS_DIR.name in cache_file.name):
                os.remove(cache_file)

conversion_successful = convert_coco_json_to_yolo_txt(ABS_JSON_ANNOTATION_FILE, ABS_IMAGES_DIR, ABS_LABELS_DIR, COCO_CLASSES)

ABS_DATASET_YAML_PATH = f"/content/{DATASET_YAML_NAME}"
if conversion_successful:
    yaml_content = f"path: {DATASET_ROOT_DIR}\ntrain: {IMAGES_SUBDIR_RELATIVE}\nval: {IMAGES_SUBDIR_RELATIVE}\n\nnames:\n"
    for i, name in enumerate(COCO_CLASSES): yaml_content += f"  {i}: {name}\n"
    try:
        with open(ABS_DATASET_YAML_PATH, 'w') as f: f.write(yaml_content)
        if not Path(ABS_DATASET_YAML_PATH).is_file(): ABS_DATASET_YAML_PATH = None
    except Exception as e: print(f"ERROR writing YAML: {e}"); ABS_DATASET_YAML_PATH = None
else:
    print("ERROR: Label conversion failed. YAML not created."); ABS_DATASET_YAML_PATH = None

if not (ABS_DATASET_YAML_PATH and Path(ABS_DATASET_YAML_PATH).exists()):
    print("FATAL: Dataset YAML creation failed or path invalid. Evaluation cannot proceed.")
else:
    print(f"Dataset YAML ready: {ABS_DATASET_YAML_PATH}")

In [ ]:
results_summary = {}
yolo_base_model = None

class ONNXCOCOCalibrationDataReader(CalibrationDataReader):
    def __init__(self, dataset_yaml_path, model_input_name, num_calibration_images=128, input_size=(640, 640)):
        self.model_input_name = model_input_name
        self.num_calibration_images = num_calibration_images
        self.input_size = input_size
        self.image_files = self._load_image_paths_from_yaml(dataset_yaml_path)
        if len(self.image_files) > self.num_calibration_images:
            selected_indices = np.random.choice(len(self.image_files), self.num_calibration_images, replace=False)
            self.image_files = [self.image_files[i] for i in selected_indices]
        elif not self.image_files: raise ValueError("No training images for ONNX calibration.")
        self.data_iter = iter(self._preprocess_images())

    def _load_image_paths_from_yaml(self, yaml_path):
        import yaml
        with open(yaml_path, 'r') as f: data = yaml.safe_load(f)
        dataset_root = Path(data.get('path', Path(yaml_path).parent))
        train_images_rel_path = data.get('train')
        if not train_images_rel_path: return []
        abs_train_dir = dataset_root / train_images_rel_path
        img_paths = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tif', '*.tiff']:
            img_paths.extend(list(abs_train_dir.glob(ext)))
        return img_paths

    def _preprocess_images(self):
        from PIL import Image
        for image_path in tqdm(self.image_files, desc="Preprocessing ONNX calib", leave=False):
            try:
                img = Image.open(image_path).convert('RGB').resize(self.input_size[::-1], Image.LANCZOS)
                img_np = (np.array(img, dtype=np.float32) / 255.0).transpose(2, 0, 1)
                yield {self.model_input_name: img_np[np.newaxis, ...]}
            except Exception: continue
    def get_next(self): return next(self.data_iter, None)
    def rewind(self): self.data_iter = iter(self._preprocess_images())

def validate_and_store_results(model_to_eval, model_id_str, data_yaml_path, device_str='cpu'):
    global results_summary
    metrics_data = {
        "mAP50-95": float('nan'), "mAP50": float('nan'), "mAP75": float('nan'),
        "Precision": float('nan'), "Recall": float('nan'), "F1": float('nan')
    }
    try:
        model = YOLO(str(model_to_eval)) if isinstance(model_to_eval, (str, Path)) else model_to_eval
        m_obj = model.val(data=data_yaml_path, split='val', device=device_str, plots=False, batch=1, verbose=False)

        if m_obj and hasattr(m_obj, 'box') and hasattr(m_obj.box, 'map'):
            metrics_data.update({"mAP50-95": m_obj.box.map, "mAP50": m_obj.box.map50, "mAP75": m_obj.box.map75})
            for metric_key, ultralytics_key in [("Precision", 'p'), ("Recall", 'r'), ("F1", 'f1')]:
                if hasattr(m_obj.box, ultralytics_key):
                    val = getattr(m_obj.box, ultralytics_key)
                    metrics_data[metric_key] = np.mean(val) if isinstance(val, np.ndarray) and val.size > 0 else val

        print(f"Metrics for {model_id_str}: mAP50={metrics_data['mAP50']:.4f}, P={metrics_data['Precision']:.4f}, R={metrics_data['Recall']:.4f}, F1={metrics_data['F1']:.4f}")

    except Exception as e:
        print(f"ERROR validating {model_id_str}: {e}")
        for key in ["mAP50-95", "mAP50", "mAP75", "Precision", "Recall", "F1"]:
            metrics_data.setdefault(key, float('nan'))

    results_summary[model_id_str] = metrics_data

In [ ]:
print("\n--- Overall Performance Summary (val2017 on CPU) ---")
header = (f"{'Model':<30} | {'mAP@.5':<10} | {'mAP@.75':<10} | {'mAP@.5-.95':<12} | "
          f"{'Precision':<10} | {'Recall':<10} | {'F1-score':<10}")
separator = "-" * len(header)
print(header)
print(separator)

model_order = ["PyTorch FP32", "ONNX FP32", "ONNX INT8"]
evaluated_models_in_order = [m for m in model_order if m in results_summary]
remaining_models = [m for m in results_summary if m not in evaluated_models_in_order]
final_print_order = evaluated_models_in_order + remaining_models

for model_name in final_print_order:
    metrics = results_summary.get(model_name, {})
    map_val = metrics.get("mAP50-95", float('nan'))
    map50_val = metrics.get("mAP50", float('nan'))
    map75_val = metrics.get("mAP75", float('nan'))
    precision_val = metrics.get("Precision", float('nan'))
    recall_val = metrics.get("Recall", float('nan'))
    f1_val = metrics.get("F1", float('nan'))

    print(f"{model_name:<30} | {map50_val:<10.4f} | {map75_val:<10.4f} | {map_val:<12.4f} | "
          f"{precision_val:<10.4f} | {recall_val:<10.4f} | {f1_val:<10.4f}")

if not results_summary: print("No results to display.")